In [7]:
    # from selenium import webdriver
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# import os

In [8]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import os

In [9]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [10]:
seasons = list(range(2021,2024))
seasons

[2021, 2022, 2023]

In [11]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [12]:
for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    filter_div = soup.find('div', class_='filter')
    links = filter_div.find_all('a')
    urls = [link.get("href") for link in links]

In [13]:
table = soup.find('table', id='schedule')

links = table.find_all("a")
hrefs = [link.get('href') for link in links]
box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link]
box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

In [14]:
box_score = box_scores[0]

browser.visit(box_score)

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
score_table = soup.find('table', id='line_score')
tbody = score_table.find('tbody')
rows = tbody.find_all('tr')

line_score = []

for row in rows:
    # Get team name
    team = row.find('th', class_='center').text
    columns = row.find_all('td')
    
    # Create line score dictionary
    if(columns !=[]):
        total = columns[4].text
    
    # Create dictionary for dataframe later
    line_score_dict = { "team": team,
                        "total": total
    }
    
    # Add dictionary to array
    line_score.append(line_score_dict)

In [16]:
# Create Data frame
import pandas as pd

df = pd.DataFrame(line_score)
df

,team,total
0,PHI,117
1,BOS,126


In [46]:
# Get basic and advanced box scores
teams = []
for i in range(len(line_score)):
    teams.append(line_score[i]['team'])
teams

['PHI', 'BOS']

In [82]:
# stats = ['basic', 'advanced']

# home_team_basic = []
# home_team_advanced = []
# away_team_basic = []
# away_team_advanced = []

# for team in teams:
#     for stat in stats:
#         # if stat == 'basic':
#         stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
#         for row in stat_table.find_all('tr'):
#             row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
#             basic.append(row_data)
#         else:
#             stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
#             for row in stat_table.find_all('tr'):
#                 row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
#                 advanced.append(row_data)

In [83]:
pd.DataFrame(basic)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,Basic Box Score Stats,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
2,Tyrese Maxey,38:11,8,16,.500,2,5,.400,3,3,...,0,1,1,2,2,0,1,5,21,-6
3,James Harden,37:17,9,14,.643,5,9,.556,12,12,...,0,8,8,7,0,0,3,3,35,+1
4,Joel Embiid,37:16,9,18,.500,1,6,.167,7,9,...,1,14,15,5,0,1,6,4,26,-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,Justin Jackson,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
329,Mfiondu Kabengele,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
330,Luke Kornet,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
331,Payton Pritchard,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [72]:
stats = ['basic', 'advanced']
home_team_tables = {'basic': [], 'advanced': []}
away_team_tables = {'basic': [], 'advanced': []}

for team in teams:
    for stat in stats:
        stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
        table_data = []
        if stat_table:
            for row in stat_table.find_all('tr'):
                row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)
                
        # Store table data on the type of stat team
        if team == teams[0]:
            home_team_tables[stat].extend(table_data)
        elif team == teams[1]:
            away_team_tables[stat].extend(table_data)

In [81]:
home_team_tables['advanced'][-1]

['Team Totals',
 '240',
 '.634',
 '.581',
 '.425',
 '.350',
 '11.8',
 '81.8',
 '46.3',
 '40.0',
 '8.1',
 '6.4',
 '13.2',
 '100.0',
 '119.2',
 '128.3',
 '']

In [33]:
# summaries = []
# basic = []
# advanced = []

# # add 'advanced' after
# stats = ['basic', 'advanced']
# for team in teams:
#     for stat in stats:
#         if stat == 'basic':
#             stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
#             tbody = stat_table.find('tbody')
#             thead = stat_table.find('thead')
            
#             # Find headers in table 
#             headers = [header.text for header in thead.find_all('tr')[1].find_all('th')]
            
#             # Find rows of table body
#             rows = tbody.find_all('tr')
#             basic = []
            
#             # Loop through rows for data
#             for row in rows:
#                 player_link = row.find('a')
#                 if player_link:
#                     player = row.find('th').text
#                     columns = row.find_all('td')
                    
#                     if len(columns) >=2:
#                     # Create basic stat dictionary
#                         MP = columns[0].text
#                         FG = columns[1].text
#                         FGA = columns[2].text
#                         FG_pct = columns[3].text
#                         three_pts = columns[4].text
#                         three_pt_att = columns[5].text
#                         three_pt_pct = columns[6].text
#                         FT = columns[7].text
#                         FTA = columns[8].text
#                         FT_pct = columns[9].text
#                         ORB = columns[10].text
#                         DRB = columns[11].text
#                         TRB = columns[12].text
#                         AST = columns[13].text
#                         STL = columns[14].text
#                         BLK = columns[15].text
#                         TOV = columns[16].text
#                         PF = columns[17].text
#                         PTS = columns[18].text
#                         DIFF = columns[19].text
                        
#                 # Create dictionary for dataframe later
#                 basic_dict = { "player": player,
#                                 "MP": MP,
#                                 "FG": FG,
#                                 "FGA": FGA,
#                               "FG%": FG_pct,
#                               "3P": three_pts,
#                               "3PA": three_pt_att,
#                               "3P%" :three_pt_pct,
#                               "FT": FT,
#                               "FTA": FTA,
#                               "FT%": FT_pct,
#                               "ORB": ORB,
#                               "DRB": DRB,
#                               "TRB": TRB,
#                               "AST": AST,
#                               "STL": STL,
#                               "BLK": BLK,
#                               "TOV": TOV,
#                               "PF": PF,
#                               "PTS": PTS,
#                               "+/-" : DIFF
#                 }

#                 # Add dictionary to array
#                 basic.append(basic_dict)
#         else:
#             stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
#             tbody = stat_table.find('tbody')
#             thead = stat_table.find('thead')
            
#             # Find headers in table 
#             headers = [header.text for header in thead.find_all('tr')[1].find_all('th')]
            
#             # Find rows of table body
#             rows = tbody.find_all('tr')
            
#             # Loop through rows for data
#             for row in rows:
#                 player_link = row.find('a')
#                 if player_link:
#                     player = player_link.text
#                     columns = row.find_all('td')
                    
#                     if len(columns) >=2:
                        
#                     # Create basic stat dictionary
#                         MP = columns[0].text
#                         TS = columns[1].text
#                         eFG = columns[2].text
#                         three_PAr = columns[3].text
#                         FTr = columns[4].text
#                         ORB_pct = columns[5].text
#                         DRB_pct = columns[6].text
#                         TRB_pct = columns[7].text
#                         AST_pct = columns[8].text
#                         STL_pct = columns[9].text
#                         BLK_pct = columns[10].text
#                         TOV_pct = columns[11].text
#                         USG_pct = columns[12].text
#                         ORtg = columns[13].text
#                         DRtg = columns[14].text
#                         BPM = columns[15].text
                        
#                 # Create dictionary for dataframe later
#                 advanced_dict = { "player": player,
#                                 "MP": MP,
#                                 "TS%": TS,
#                                 "eFG": eFG,
#                                 "3PAr": three_PAr, 
#                                 "FTr" : FTr,
#                                 "ORB%": ORB_pct,   
#                                 "DRB%" : DRB_pct,  
#                                 "TRB%": TRB_pct,   
#                                 "AST%" : AST_pct,  
#                                 "STL%" : STL_pct,  
#                                 "BLK%": BLK_pct,   
#                                 "TOV%": TOV_pct,  
#                                 "USG%" : USG_pct, 
#                                 "ORtg" : ORtg, 
#                                 "DRtg": DRtg,  
#                                 "BPM": BPM  
#                 }

#                 # Add dictionary to array
#                 advanced.append(advanced_dict)
           


In [39]:
stat_table = soup.find('table', id=f"box-BOS-game-basic")

data = []
for row in stat_table.find_all('tr'):
    row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
    data.append(row_data)

In [40]:
data

[['', 'Basic Box Score Stats'],
 ['Starters',
  'MP',
  'FG',
  'FGA',
  'FG%',
  '3P',
  '3PA',
  '3P%',
  'FT',
  'FTA',
  'FT%',
  'ORB',
  'DRB',
  'TRB',
  'AST',
  'STL',
  'BLK',
  'TOV',
  'PF',
  'PTS',
  '+/-'],
 ['Jayson Tatum',
  '38:38',
  '13',
  '20',
  '.650',
  '2',
  '7',
  '.286',
  '7',
  '9',
  '.778',
  '0',
  '12',
  '12',
  '4',
  '1',
  '1',
  '3',
  '1',
  '35',
  '+10'],
 ['Jaylen Brown',
  '38:34',
  '14',
  '24',
  '.583',
  '4',
  '12',
  '.333',
  '3',
  '3',
  '1.000',
  '0',
  '3',
  '3',
  '2',
  '2',
  '1',
  '4',
  '1',
  '35',
  '+7'],
 ['Marcus Smart',
  '36:05',
  '3',
  '8',
  '.375',
  '1',
  '4',
  '.250',
  '7',
  '8',
  '.875',
  '1',
  '2',
  '3',
  '7',
  '1',
  '0',
  '1',
  '3',
  '14',
  '0'],
 ['Derrick White',
  '24:02',
  '1',
  '3',
  '.333',
  '0',
  '1',
  '.000',
  '0',
  '1',
  '.000',
  '1',
  '2',
  '3',
  '4',
  '1',
  '0',
  '1',
  '2',
  '2',
  '+9'],
 ['Al Horford',
  '23:06',
  '2',
  '7',
  '.286',
  '2',
  '5',
  '.400',

In [34]:
# Create Data frame
import pandas as pd

basic_df = pd.DataFrame(basic)
advanced_df = pd.DataFrame(advanced)

basic_df
advanced_df

,player,MP,TS%,eFG,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,Tyrese Maxey,38:11,.606,.563,.313,.188,0.0,3.8,1.9,8.4,2.6,0.0,5.5,21.7,118,129,-0.9
1,James Harden,37:17,.908,.821,.643,.857,0.0,31.2,15.4,31.7,0.0,0.0,13.5,27.0,165,132,14.5
2,Joel Embiid,37:16,.592,.528,.333,.500,3.8,54.6,28.8,22.7,0.0,2.7,21.5,33.9,103,127,-0.2
3,Tobias Harris,34:14,.605,.607,.429,.143,4.1,4.2,4.2,0.0,4.3,0.0,0.0,19.6,122,125,2.7
4,P.J. Tucker,33:01,.600,.600,.400,.000,8.6,8.8,8.7,0.0,0.0,3.1,28.6,9.6,98,132,-4.2
5,P.J. Tucker,33:01,.600,.600,.400,.000,8.6,8.8,8.7,0.0,0.0,3.1,28.6,9.6,98,132,-4.2
6,De'Anthony Melton,20:33,.625,.625,.500,.000,0.0,0.0,0.0,0.0,2.4,0.0,0.0,8.8,122,130,-2.0
7,Danuel House Jr.,16:12,.174,.000,.500,1.000,0.0,9.0,4.4,7.4,6.0,0.0,25.8,10.8,43,121,-5.6
8,Georges Niang,12:09,.375,.375,.750,.000,0.0,0.0,0.0,11.0,0.0,0.0,0.0,14.9,88,136,-6.6
9,Montrezl Harrell,10:44,.333,.333,.000,.000,0.0,0.0,0.0,0.0,0.0,9.5,25.0,16.8,49,128,-16.3
